In [8]:
using ParallelTemperingMonteCarlo
using BenchmarkTools,Plots,StaticArrays
using Random

In [9]:
#set random seed - for reproducibility
Random.seed!(1234)

# number of atoms
n_atoms = 13

# temperature grid
ti = 5.
tf = 16.
n_traj = 32

temp = TempGrid{n_traj}(ti,tf) 

# MC simulation details

mc_cycles = 1000000 #default 20% equilibration cycles on top

mc_sample = 1  #sample every mc_sample MC cycles

#move_atom=AtomMove(n_atoms) #move strategy (here only atom moves, n_atoms per MC cycle)
displ_atom = 0.1 # Angstrom
n_adjust = 100

max_displ_atom = [0.1*sqrt(displ_atom*temp.t_grid[i]) for i in 1:n_traj]

mc_params = MCParams(mc_cycles, n_traj, n_atoms, mc_sample = mc_sample, n_adjust = n_adjust)

#moves - allowed at present: atom, volume and rotation moves (volume,rotation not yet implemented)
move_strat = MoveStrategy(atom_moves = n_atoms)  

#ensemble
ensemble = NVT(n_atoms)

#ELJpotential for neon
#c1=[-10.5097942564988, 0., 989.725135614556, 0., -101383.865938807, 0., 3918846.12841668, 0., -56234083.4334278, 0., 288738837.441765]
#elj_ne1 = ELJPotential{11}(c1)

c=[-10.5097942564988, 989.725135614556, -101383.865938807, 3918846.12841668, -56234083.4334278, 288738837.441765]
pot = ELJPotentialEven{6}(c)

#starting configurations
#icosahedral ground state of Ne13 (from Cambridge cluster database) in Angstrom
pos_ne13 = [[2.825384495892464, 0.928562467914040, 0.505520149314310],
[2.023342172678102,	-2.136126268595355, 0.666071287554958],
[2.033761811732818,	-0.643989413759464, -2.133000349161121],
[0.979777205108572,	2.312002562803556, -1.671909307631893],
[0.962914279874254,	-0.102326586625353, 2.857083360096907],
[0.317957619634043,	2.646768968413408, 1.412132053672896],
[-2.825388342924982, -0.928563755928189, -0.505520471387560],
[-0.317955944853142, -2.646769840660271, -1.412131825293682],
[-0.979776174195320, -2.312003751825495, 1.671909138648006],
[-0.962916072888105, 0.102326392265998,	-2.857083272537599],
[-2.023340541398004, 2.136128558801072,	-0.666071089291685],
[-2.033762834001679, 0.643989905095452, 2.132999911364582],
[0.000002325340981,	0.000000762100600, 0.000000414930733]]

#convert to Bohr
AtoBohr = 1.8897259886
pos_ne13 = pos_ne13 * AtoBohr

length(pos_ne13) == n_atoms || error("number of atoms and positions not the same - check starting config")

#boundary conditions 
bc_ne13 = SphericalBC(radius=5.32*AtoBohr)   #5.32 Angstrom

#starting configuration
start_config = Config(pos_ne13, bc_ne13)

#histogram information
n_bin = 100
#en_min = -0.006    #might want to update after equilibration run if generated on the fly
#en_max = -0.001    #otherwise will be determined after run as min/max of sampled energies (ham vector)

#construct array of MCState (for each temperature)
mc_states = [MCState(temp.t_grid[i], temp.beta_grid[i], start_config, pot) for i in 1:n_traj]

#results = Output(n_bin, max_displ_vec)
results = Output{Float64}(n_bin; en_min = mc_states[1].en_tot)


Output{Float64}(100, 0.0, 0.0, Float64[], Float64[], Float64[], Vector{Float64}[], Vector{Float64}[], Float64[], Float64[], Float64[], Float64[])

In [10]:
ptmc_run!(mc_states, move_strat, mc_params, pot, ensemble, results; save_ham = false)


Total number of moves per MC cycle: 13



equilibration done


MC loop done.
[0.0003234110423948385, 0.00034130435146749016, 0.0003571979730485569, 0.0003751826872688714, 0.00038929116332445156, 0.00041586364789964576, 0.00043669694264594094, 0.0004595895593319723, 0.0004890435704389878, 0.0005157634753972615, 0.0005536751008066922, 0.0005723147050125483, 0.0006169481887285358, 0.000683777774995162, 0.0007252243017238607, 0.000822837124956102, 0.0008936290651810269, 0.0009485259670130599, 0.0011550216114052253, 0.001486674414643203, 0.0017357931127196526, 0.0022538799773156406, 0.002685202167626738, 0.0030124683418126814, 0.003227185612361041, 0.0031974041582545276, 0.003076489630752461, 0.002817309162204895, 0.0025391775980580186, 0.0022260287657100224, 0.002074304958103143, 0.0020101649064780125]
done


In [12]:
delta_en_hist = (results.en_max - results.en_min) / (results.n_bin - 1)
delta_r2 = delta_r2 = 4*start_config.bc.radius2/results.n_bin/5

0.8085575246693824

In [21]:
@benchmark ptmc_cycle!($mc_states,$results,move_strat,mc_params,pot,ensemble,13,13,0,0,true,1000,pwd(),delta_en_hist,delta_r2)

BenchmarkTools.Trial: 1499 samples with 1 evaluation.
 Range (min … max):  1.939 ms … 11.338 ms  ┊ GC (min … max):  0.00% … 61.63%
 Time  (median):     3.176 ms              ┊ GC (median):     0.00%
 Time  (mean ± σ):   3.330 ms ±  1.771 ms  ┊ GC (mean ± σ):  13.69% ± 17.59%

  █▇▃         ▃█▆▃                                            
  █████▇▇████▇█████▆▇▅▁▅▁▁▄▁▄▁▁▄▁▄▄▁▁▁▄▁▁▁▁▁▄▆▅▇▇█▇▇▇█▇██▇▇▇ █
  1.94 ms      Histogram: log(frequency) by time     9.07 ms <

 Memory estimate: 9.15 MiB, allocs estimate: 47302.

Old versions

In [14]:
"""    sampling_step(mc_params, mc_states, save_index, saveham::Bool)
A function to store the information at the end of an MC_Cycle, replacing the manual if statements previously in PTMC_run. 
"""
function sampling_step(mc_params,mc_states,save_index, saveham::Bool)  
        if rem(save_index, mc_params.mc_sample) == 0
            for indx_traj=1:mc_params.n_traj
                if saveham == true
                    push!(mc_states[indx_traj].ham, mc_states[indx_traj].en_tot) #to build up ham vector of sampled energies
                else
                    mc_states[indx_traj].ham[1] += mc_states[indx_traj].en_tot
                    #add E,E**2 to the correct positions in the hamiltonian
                    mc_states[indx_traj].ham[2] += (mc_states[indx_traj].en_tot*mc_states[indx_traj].en_tot)
                end
            end
        end 
end
# """

#     initialise_histograms!(mc_params,results,T)
# functionalised the step in which we build the energy histograms  
# """
# function initialise_histograms!(mc_params,mc_states,results; full_ham = true,e_bounds = [0,0])    
#     T = typeof(mc_states[1].en_tot)
#     en_min = T[]
#     en_max = T[]

#     r_max = 4*mc_states[1].config.bc.radius2 #we will work in d^2
#     delta_r = r_max/results.n_bin/5 #we want more bins for the RDFs

#     if full_ham == true
#         for i_traj in 1:mc_params.n_traj
#             push!(en_min,minimum(mc_states[i_traj].ham))
#             push!(en_max,maximum(mc_states[i_traj].ham))
#         end
    
#         global_en_min = minimum(en_min)
#         global_en_max = maximum(en_max)
#     else

#         #we'll give ourselves a 6% leeway here
#         global_en_min = e_bounds[1] - abs(0.03*e_bounds[1])
#         global_en_max = e_bounds[2] + abs(0.03*e_bounds[2])
#     end

#     for i_traj = 1:mc_params.n_traj
#         histogram = zeros(results.n_bin + 2)
#         push!(results.en_histogram, histogram)
#         RDF = zeros(results.n_bin*5)
#         push!(results.rdf,RDF)
#     end
    

#     delta_en_hist = (global_en_max - global_en_min) / (results.n_bin - 1)


#     results.en_min = global_en_min
#     results.en_max = global_en_max

  
#         return  delta_en_hist
# end
"""
    updaterdf!(mc_states,results,delta_r2)
For each state in a vector of mc_states, we use the distance squared matrix to determine which bin (between zero and 2*r_bound) the distance falls into, we then update results.rdf[bin] to build the radial distribution function
"""
function updaterdf!(mc_states,results,delta_r2)
    for j_traj in eachindex(mc_states)
        for element in mc_states[j_traj].dist2_mat 
            rdf_index=floor(Int,(element/delta_r2))
            if rdf_index != 0
                results.rdf[j_traj][rdf_index] +=1
            end
        end
    end
end
"""
    updatehistogram!(mc_params,mc_states,results,delta_en_hist ; fullham=true)
Performed either at the end or during the mc run according to fullham=true/false (saved all datapoints or calculated on the fly). Uses the energy bounds and the previously defined delta_en_hist to calculate the bin in which te current energy value falls for each trajectory. This is used to build up the energy histograms for post-analysis.
"""
function updatehistogram!(mc_params,mc_states,results,delta_en_hist ; fullham=true)

    for update_traj_index in 1:mc_params.n_traj
        
        if fullham == true #this is done at the end of the cycle

            hist = zeros(results.n_bin)#EnHist(results.n_bin, global_en_min, global_en_max)
            for en in mc_states[update_traj_index].ham
                hist_index = floor(Int,(en - results.en_min) / delta_en_hist) + 1
                hist[hist_index] += 1

            end
        push!(results.en_histogram, hist)

        else #this is done throughout the simulation

            en = mc_states[update_traj_index].en_tot

            hist_index = floor(Int,(en - results.en_min) / delta_en_hist) + 1 

            if hist_index < 1 #if energy too low
                results.en_histogram[update_traj_index][1] += 1 #add to place 1
            elseif hist_index > results.n_bin #if energy too high
                results.en_histogram[update_traj_index][(results.n_bin +2)] += 1 #add to place n_bin +2
            else
                results.en_histogram[update_traj_index][(hist_index+1)] += 1
            end

        end
    end

end

updatehistogram!

In [19]:
function test_ptmc_cycle!(mc_states,results,move_strat, mc_params, pot, ensemble ,n_steps ,a ,v ,r, save_ham, save, i,save_dir ;delta_en_hist=0.)


    mc_states = mc_cycle!(mc_states, move_strat, mc_params, pot,  ensemble, n_steps, a, v, r) 
    #sampling step
    sampling_step(mc_params,mc_states,i,save_ham)

    if save_ham == false
        updatehistogram!(mc_params,mc_states,results,delta_en_hist,fullham=save_ham)
        updaterdf!(mc_states,results,(4*mc_states[1].config.bc.radius2/(results.n_bin*5)))

    end

    #step adjustment
    if rem(i, mc_params.n_adjust) == 0
        for i_traj = 1:mc_params.n_traj
            update_max_stepsize!(mc_states[i_traj], mc_params.n_adjust, a, v, r)
        end 
    end

    if save == true
        if rem(i,1000) == 0

            save_states(mc_params,mc_states,i,save_dir)
            if save_ham == false
                save_results(results,save_dir)
            end
        end
    end

end

test_ptmc_cycle! (generic function with 1 method)

In [22]:
@benchmark test_ptmc_cycle!($mc_states,$results,move_strat,mc_params,pot,ensemble,13,13,0,0,false,true,1000,pwd(),delta_en_hist = delta_en_hist)

BenchmarkTools.Trial: 1408 samples with 1 evaluation.
 Range (min … max):  1.950 ms … 10.336 ms  ┊ GC (min … max):  0.00% … 50.61%
 Time  (median):     3.462 ms              ┊ GC (median):     0.00%
 Time  (mean ± σ):   3.546 ms ±  1.845 ms  ┊ GC (mean ± σ):  13.39% ± 17.55%

  █▇▄▃▃▃▂▁ ▁▂▃▇▆▄▄▃▃▂▁   ▁                                   ▁
  ████████████████████▆▆██▆▁▅▁▄▄▄▄▁▁▁▁▄▁▁▁▁▆▆▅▄█▅▇▇▇▇█▇▇█▇▆▇ █
  1.95 ms      Histogram: log(frequency) by time     9.56 ms <

 Memory estimate: 9.15 MiB, allocs estimate: 47304.